<h1>NBA Shot predictor</h1>

In [4]:
#Set up the library
import pandas as pd

In [5]:
#Set up the dataframe
df = pd.read_csv('data/NBA Shot Locations 1997 - 2020.csv')

In [17]:
#Test Data Frame will be Kyle Lowry, Raptors Player, PG
test_df=df[df['Player Name'] == 'Kyle Lowry']
test_df.head(3)

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Action Type,...,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Shot Made Flag,Game Date,Home Team,Away Team,Season Type
1732363,20600010,336,200768,Kyle Lowry,1610612763,Memphis Grizzlies,3,1,43,Jump Shot,...,Left Side Center(LC),16-24 ft.,21,-136,164,1,20061101,MEM,NYK,Regular Season
1732364,20600010,467,200768,Kyle Lowry,1610612763,Memphis Grizzlies,4,3,6,Layup Shot,...,Center(C),Less Than 8 ft.,0,0,0,0,20061101,MEM,NYK,Regular Season
1732365,20600010,503,200768,Kyle Lowry,1610612763,Memphis Grizzlies,4,0,21,Driving Layup Shot,...,Center(C),Less Than 8 ft.,0,0,0,0,20061101,MEM,NYK,Regular Season


## EDA

In [16]:
df['Shot Zone Basic'].nunique

AttributeError: 'function' object has no attribute 'count'

In [6]:
df.groupby(['Player ID']).sum()

,Game ID,Game Event ID,Team ID,Period,Minutes Remaining,Seconds Remaining,Shot Distance,X Location,Y Location,Shot Made Flag,Game Date
Player ID,,,,,,,,,,,
3,39242876973,348329,2430414646129,3828,7869,44032,17174,-4630,85324,643,30183858684
7,11932981383,80830,645855713953,964,2571,11528,4425,-10379,27044,166,8011682735
12,208600892,2774,11274289334,26,32,186,67,0,449,2,139931546
15,91008037308,855518,6025302281464,9339,20494,107301,61045,-21843,356813,1621,74854951169
17,34832524061,244133,1815160563615,2671,5505,32913,14233,-27684,90386,473,22514284987
...,...,...,...,...,...,...,...,...,...,...,...
1629744,2430961478,42250,178778016471,312,611,3621,2282,-986,14366,54,2241885448
1629745,109503757,2128,8053063780,14,26,168,71,288,431,1,101001018
1629750,2146243339,40094,157840048324,280,438,2956,760,1053,4646,49,1979128844


Roll up the classes 
Deep Learn = can predict x and y